In [ ]:
# %%
"""
Jupyter notebook cell 0
Подготовка окружения
"""
import os, json, math, random, pathlib, itertools, textwrap
import numpy as np
import pandas as pd

import torch
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances

import matplotlib.pyplot as plt
import seaborn as sns         # только для красивой тепловой карты
from tqdm.auto import tqdm

# %%
"""
cell 1
Читаем данные
"""
DATA_PATH = "../logs_with_labels.csv"     # путь к csv
df = pd.read_csv(DATA_PATH)
df["id"] = df.index                       # явный id, чтобы не потерять строки
texts = df["errors"].astype(str).tolist() # список строковых сообщений
N = len(texts)
print(f"Количество строк: {N}")

# %%
"""
cell 2
Каталоги для результатов
"""
EMB_DIR       = pathlib.Path("embeddings") ; EMB_DIR.mkdir(exist_ok=True)
DIST_DIR_ROOT = pathlib.Path("distances")  ; DIST_DIR_ROOT.mkdir(exist_ok=True)
HM_DIR        = pathlib.Path("heatmaps")   ; HM_DIR.mkdir(exist_ok=True)

# %%
"""
cell 3
Список моделей. Добавляйте / убирайте по необходимости.
Ключ словаря — читабельное имя, значение — id в HuggingFace Hub.
"""
MODELS = {
    "all-MiniLM-L6-v2":   "sentence-transformers/all-MiniLM-L6-v2",
    "all-mpnet-base-v2":  "sentence-transformers/all-mpnet-base-v2",
    "bge-base-en-v1.5":   "BAAI/bge-base-en-v1.5",
    "bge-large-en-v1.5":  "BAAI/bge-large-en-v1.5",
    "roberta-large":     "roberta-large",
    "deberta-v3-large":  "microsoft/deberta-v3-large",
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH  = 32                         # подстройте под память

# %%
"""
cell 4
Функция для быстрой генерации эмбеддингов через SentenceTransformers
normalize_embeddings=True сразу даёт l2-нормированные вектора — удобно для косинуса.
"""
def build_embeddings(model_id: str, texts, batch_size=BATCH):
    encoder = SentenceTransformer(model_id, device=DEVICE)
    emb = encoder.encode(
        texts,
        batch_size           = batch_size,
        show_progress_bar    = True,
        convert_to_numpy     = True,
        normalize_embeddings = True
    )
    return emb.astype(np.float32)

# %%
"""
cell 5
Основной цикл по моделям
"""
for name, hub_id in MODELS.items():
    print(f"\n=== {name} ===")
    # 1) эмбеддинги
    emb_path = EMB_DIR / f"{name}.npy"
    if emb_path.exists():
        emb = np.load(emb_path)
        print("✓ Embeddings загружены из кеша")
    else:
        emb = build_embeddings(hub_id, texts)
        np.save(emb_path, emb)
        print("✓ Embeddings сохранены:", emb.shape)
    
    # 2) матрица косинусных расстояний
    dist_dir = DIST_DIR_ROOT / name ; dist_dir.mkdir(exist_ok=True)
    dist_csv = dist_dir / "cosine_distances.csv"
    if dist_csv.exists():
        print("✓ Distance-matrix уже есть, пропускаю расчёт")
        dists = pd.read_csv(dist_csv, index_col=0).values
    else:
        print("… считаю матрицу расстояний (это O(N²) ≈ %.1f М)" % (N**2/1e6))
        dists = cosine_distances(emb)           # shape (N, N)
        # сохраняем CSV — строки и столбцы пронумерованы id
        pd.DataFrame(dists, index=df.id, columns=df.id)\
          .to_csv(dist_csv, index_label="id")
        print("✓ Distance-matrix сохранена:", dist_csv)
    
    # 3) тепловая карта
    hm_path = HM_DIR / f"{name}.png"
    if not hm_path.exists():
        print("… рисую heatmap")
        plt.figure(figsize=(8, 7))
        sns.heatmap(dists, cmap="viridis", cbar_kws=dict(label="cosine distance"),
                    xticklabels=False, yticklabels=False)
        plt.title(f"Cosine-distance heatmap — {name}")
        plt.tight_layout()
        plt.savefig(hm_path, dpi=150)
        plt.close()
        print("✓ Heatmap сохранён:", hm_path)
    else:
        print("✓ Heatmap уже есть")

print("\nГотово ✅")


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-17 05:35:51.243613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-17 05:35:59.048041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Количество строк: 375

=== all-MiniLM-L6-v2 ===


Batches: 100%|██████████| 12/12 [00:05<00:00,  2.08it/s]


✓ Embeddings сохранены: (375, 384)
… считаю матрицу расстояний (это O(N²) ≈ 0.1 М)
✓ Distance-matrix сохранена: distances/all-MiniLM-L6-v2/cosine_distances.csv
… рисую heatmap
✓ Heatmap сохранён: heatmaps/all-MiniLM-L6-v2.png

=== all-mpnet-base-v2 ===


Batches: 100%|██████████| 12/12 [00:04<00:00,  2.68it/s]


...

No sentence-transformers model found with name roberta-large. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Batches: 100%|██████████| 12/12 [00:11<00:00,  1.04it/s]


✓ Embeddings сохранены: (375, 1024)
… считаю матрицу расстояний (это O(N²) ≈ 0.1 М)
✓ Distance-matrix сохранена: distances/roberta-large/cosine_distances.csv
… рисую heatmap


No sentence-transformers model found with name microsoft/deberta-v3-large. Creating a new one with mean pooling.


✓ Heatmap сохранён: heatmaps/roberta-large.png

=== deberta-v3-large ===


/home/jupyter/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Batches: 100%|██████████| 12/12 [00:18<00:00,  1.52s/it]